In [1]:
import cv2
import pandas as pd
import os
import csv
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import Dependency

# Hospitalization, ICU, and CFR computation

In [ ]:
def augment_ontario_public_health(lst):
    augment = np.zeros(shape=(100,))
    augment[0:12] = np.array([lst[0]] * 12)
    augment[12:20] = np.array([lst[1]] * 8)
    augment[20:30] = np.array([lst[2]] * 10)
    augment[30:40] = np.array([lst[3]] * 10)
    augment[40:50] = np.array([lst[4]] * 10)
    augment[50:60] = np.array([lst[5]] * 10)
    augment[60:70] = np.array([lst[6]] * 10)
    augment[70:80] = np.array([lst[7]] * 10)
    augment[80:] = np.array([lst[8]] * 20)
    return augment

In [ ]:
def stratification(raw=np.zeros(shape=(100, ))):
    age_stratification = np.zeros(shape=(16, ))
    for i in range(15):
        age_stratification[i] = np.sum(raw[5*i:5 * (i+1)]) / 5
    age_stratification[15] = np.sum(raw[74:]) / 25
    return age_stratification

In [ ]:
d = Dependency.Dependency()
cases_read_path = d.get_dependency_path() + 'Canada_hospitalization_inference.csv'
df = pd.read_csv(cases_read_path)
print(df)
cases = np.array(list(df.cases))
hospitalization = np.array(list(df.hospitalization))
icu = np.array(list(df.ICU))
deaths = np.array(list(df.death))

In [ ]:
cases_augment = augment_ontario_public_health(cases)
hospitalized_augment = augment_ontario_public_health(hospitalization)
icu_augment = augment_ontario_public_health(icu)
deaths_augment = augment_ontario_public_health(deaths)

In [ ]:
cases_augment = np.reshape(np.array(cv2.GaussianBlur(cases_augment.reshape(100, 1), (25, 25), 0)), newshape=(100,))
hospitalized_augment = np.reshape(np.array(cv2.GaussianBlur(hospitalized_augment.reshape(100, 1), (25, 25), 0)), newshape=(100,))
icu_augment = np.reshape(np.array(cv2.GaussianBlur(icu_augment.reshape(100, 1), (25, 25), 0)), newshape=(100,))
deaths_augment = np.reshape(np.array(cv2.GaussianBlur(deaths_augment.reshape(100, 1), (25, 25), 0)), newshape=(100,))

In [ ]:
hosp_ratio = hospitalized_augment / cases_augment
icu_ratio = icu_augment / cases_augment
cfr = deaths_augment / cases_augment

In [ ]:
plt.plot(list(range(0, 100)), hosp_ratio)
plt.show()

In [ ]:
plt.plot(list(range(0, 100)), icu_ratio)
plt.show()

In [ ]:
plt.plot(list(range(0, 100)), cfr)
plt.show()

In [ ]:
case_stratified = stratification(raw=cases_augment)
hosp_stratified = stratification(raw=hosp_ratio)
icu_stratified = stratification(raw=icu_ratio)
cfr_stratified = stratification(raw=cfr)

In [ ]:
plt.figure(figsize=(15,6))
plt.subplot()
plt.subplot(1,3,1), plt.bar(range(0, 16), hosp_stratified)
plt.subplot(1,3,2), plt.bar(range(0, 16), icu_stratified)
plt.subplot(1,3,3), plt.bar(range(0, 16), cfr_stratified)

In [ ]:
print(case_stratified / np.sum(case_stratified))
print(hosp_stratified)
print(np.min(hosp_stratified), np.max(hosp_stratified))
print(icu_stratified)
print(np.min(icu_stratified), np.max(icu_stratified))
print(cfr_stratified)
print(np.min(cfr_stratified), np.max(cfr_stratified))

In [ ]:
susceptibility_raw = np.zeros(shape=(100, ))

susceptibility_raw[0:10] = 0.4
susceptibility_raw[10:20] = 0.38
susceptibility_raw[20:30] = 0.79
susceptibility_raw[30:40] = 0.86
susceptibility_raw[40:50] = 0.8
susceptibility_raw[50:60] = 0.82
susceptibility_raw[60:70] = 0.88
susceptibility_raw[70:100] = 0.74

susceptibility = np.reshape(np.array(cv2.GaussianBlur(susceptibility_raw.reshape(100, 1), (25, 25), 0)),
                            newshape=(100,))

susceptibility_stratified = stratification(raw=susceptibility)

In [ ]:
clinical_raw = np.zeros(shape=(100, ))

clinical_raw[0:10] = 0.29
clinical_raw[10:20] = 0.21
clinical_raw[20:30] = 0.27
clinical_raw[30:40] = 0.33
clinical_raw[40:50] = 0.4
clinical_raw[50:60] = 0.49
clinical_raw[60:70] = 0.63
clinical_raw[70:100] = 0.69

clinical = np.reshape(np.array(cv2.GaussianBlur(clinical_raw.reshape(100, 1), (25, 25), 0)),
                            newshape=(100,))

clinical_stratified = stratification(raw=clinical)

In [ ]:
plt.figure(figsize=(15,6))
plt.subplot()
plt.subplot(1,2,1), plt.plot(susceptibility_raw)
plt.subplot(1,2,2), plt.plot(clinical_raw)

In [ ]:
plt.figure(figsize=(15,6))
plt.subplot()
plt.subplot(1,2,1), plt.plot(susceptibility)
plt.subplot(1,2,2), plt.plot(clinical)

In [ ]:
plt.figure(figsize=(15,6))
plt.subplot()
plt.subplot(1,2,1), plt.bar(range(0, 16), susceptibility_stratified)
plt.subplot(1,2,2), plt.bar(range(0, 16), clinical_stratified)

In [ ]:
print(susceptibility_stratified)
print(clinical_stratified)

## Fraction calculation (normalization)

In [ ]:
case_dist = cases_augment/ np.sum(cases_augment)
hosp_dist = hospitalized_augment / np.sum(hospitalized_augment)
icu_dist = icu_augment/ np.sum(icu_augment)
deaths_dist = deaths_augment / np.sum(deaths_augment)

print(sum(case_dist), sum(hosp_dist), sum(icu_dist), sum(deaths_dist))

In [ ]:
plt.figure(figsize=(15,3))
plt.subplot()
plt.subplot(1,4,1), plt.plot(case_dist)
plt.subplot(1,4,2), plt.plot(hosp_dist)
plt.subplot(1,4,3), plt.plot(icu_dist)
plt.subplot(1,4,4), plt.plot(deaths_dist)
plt.show()
print(sum(case_dist), sum(hosp_dist), sum(icu_dist), sum(deaths_dist))

In [ ]:
case_dist = stratification(case_dist)
hosp_dist = stratification(hosp_dist)
icu_dist = stratification(icu_dist)
deaths_dist = stratification(deaths_dist)

# Normalize

case_dist = case_dist / np.sum(case_dist)
hosp_dist = hosp_dist /  np.sum(hosp_dist)
icu_dist = icu_dist / np.sum(icu_dist)
deaths_dist = deaths_dist /  np.sum(deaths_dist)

print(sum(case_dist), sum(hosp_dist), sum(icu_dist), sum(deaths_dist))

In [ ]:
plt.figure(figsize=(15,3))
plt.subplot()
plt.subplot(1,4,1), plt.bar(range(0, 16), case_dist)
plt.subplot(1,4,2), plt.bar(range(0, 16), hosp_dist)
plt.subplot(1,4,3), plt.bar(range(0, 16), icu_dist)
plt.subplot(1,4,4), plt.bar(range(0, 16), deaths_dist)


In [ ]:
print(case_dist)
print(hosp_dist)
print(icu_dist)
print(deaths_dist)

In [ ]:
print(np.sum(icu) / np.sum(hospitalization))